# Titeseq barcode aggregation

This notebook reads in the barcode-level counts, barcode-variant mapping, and sequencing runs total cells, so that we can prep Titeseq data for fitting by:

1. Merging in the variants substitution annotations to barcodes.
2. Normalize the counts by the total cells in each sequencing run.
3. Aggregate the counts by the variant substitution annotations.
4. Filter out low count barcodes and variants.  

## Set up analysis

In [1]:
import warnings
import yaml
import os

import numpy as np
import pandas as pd
from IPython.display import display, HTML

Ignore warnings that clutter output:

In [2]:
warnings.simplefilter('ignore')

### Parameters for notebook
Read the configuration file:

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

Make output directory if needed:

In [4]:
os.makedirs(config['counts_dir'], exist_ok=True)

## Data

In [5]:
barcode_runs = pd.read_csv(config['barcode_runs']).drop(columns=["R1"])
barcode_runs.query("sample.str.startswith('TiteSeq') | sample.str.startswith('SortSeq')", inplace=True)
barcode_runs.set_index(["library", "sample"], inplace=True)
display(HTML(barcode_runs.head().to_html(index=True)))
print(barcode_runs.shape)

(80, 5)


In [6]:
variant_counts = pd.read_csv(config['variant_counts_file'])
variant_counts.query("sample.str.startswith('TiteSeq') | sample.str.startswith('SortSeq')", inplace=True)
display(HTML(variant_counts.head().to_html(index=True)))
print(variant_counts.shape)

,barcode,count,library,sample
0,AGCATACCCTTAACAA,26343,lib1,SortSeq_bin1
1,TGACGCCTTATCCTCC,20015,lib1,SortSeq_bin1
2,TGCGATGGTACGTCAA,15678,lib1,SortSeq_bin1
3,AACTACACGGATAGGT,14906,lib1,SortSeq_bin1
4,CATAATGAATGTGCAA,14667,lib1,SortSeq_bin1


(7697160, 4)


In [7]:
codon_variant_table = pd.read_csv(config['codon_variant_table_file'])
display(HTML(codon_variant_table.head().to_html(index=True)))
print(codon_variant_table.shape)

,target,library,barcode,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
0,CGG_naive,lib1,AAAAAAAAAACACCGG,6,GGC119GGT TTA200ACT,L200T,2,1
1,CGG_naive,lib1,AAAAAAAAAACATGAG,1,CAG16TGG,Q16W,1,1
2,CGG_naive,lib1,AAAAAAAAAAGCGACG,1,GTG156CAT,V156H,1,1
3,CGG_naive,lib1,AAAAAAAAAAGGAAAG,6,GTG110GGT,V110G,1,1
4,CGG_naive,lib1,AAAAAAAAAATATAGA,1,TAC47CCA,Y47P,1,1


(192429, 8)


Define concentrations and bins

In [8]:
concs = np.array(config['concentrations']['CGG']).astype(float)
concs

array([1.e-06, 1.e-07, 1.e-08, 1.e-09, 1.e-10, 1.e-11, 1.e-12, 1.e-13,
       0.e+00])

Combine tables to make a single barcode level frame

In [9]:
df_barcodes = variant_counts.merge(codon_variant_table, on=("barcode", "library"), how="left")
display(HTML(df_barcodes.head().to_html(index=True)))
print(df_barcodes.shape)

,barcode,count,library,sample,target,variant_call_support,codon_substitutions,aa_substitutions,n_codon_substitutions,n_aa_substitutions
0,AGCATACCCTTAACAA,26343,lib1,SortSeq_bin1,CGG_naive,65,CGT38ATT GTA148GAA,R38I V148E,2,2
1,TGACGCCTTATCCTCC,20015,lib1,SortSeq_bin1,CGG_naive,23,GAT100GTT CAG217TAG,D100V Q217*,2,2
2,TGCGATGGTACGTCAA,15678,lib1,SortSeq_bin1,CGG_naive,26,GAC72TGT,D72C,1,1
3,AACTACACGGATAGGT,14906,lib1,SortSeq_bin1,CGG_naive,31,CAA133AAA TTT137TAG,Q133K F137*,2,2
4,CATAATGAATGTGCAA,14667,lib1,SortSeq_bin1,CGG_naive,35,TAT94ATT CAG217TGG GAG232TAG,Y94I Q217W E232*,3,3


(7697160, 10)


Add concentrations by parsing the sample name.

In [10]:
# parse the sample name to get the antigen concentration and bin
df_barcodes["antigen_concentration"] = np.nan
df_barcodes.loc[df_barcodes['sample'].str.contains('TiteSeq'), "antigen_concentration"] = concs[
    df_barcodes.query(f"sample.str.contains('TiteSeq')")["sample"].str.extract(r"TiteSeq_(\d+)").astype(int) - 1
]
df_barcodes["bin"] = df_barcodes["sample"].str[-1].astype(int)

## Filter synonymous variants with silent mutations

In [11]:

synonymous = df_barcodes.query("n_aa_substitutions == 0 & n_codon_substitutions > 0")
print(f"There are {synonymous.shape[0]} synonymous variants with codon substitutions, dropping them.")
df_barcodes.query("~(n_aa_substitutions == 0 & n_codon_substitutions > 0)", inplace=True)

# drop columns that are not needed
df_barcodes.drop(columns=["codon_substitutions", "n_codon_substitutions", "target", "variant_call_support"], inplace=True)
df_barcodes.rename(columns={"aa_substitutions": "variant",
                            "count": "read_count"},
                   inplace=True)
df_barcodes = df_barcodes.loc[:, ["sample", "library", "variant", "n_aa_substitutions", "barcode", "antigen_concentration", "bin", "read_count"]]
df_barcodes.sort_values(by=list(df_barcodes.columns), inplace=True)
df_barcodes.variant = df_barcodes.variant.fillna("WT")

df_barcodes

There are 27000 synonymous variants with codon substitutions, dropping them.


,sample,library,variant,n_aa_substitutions,barcode,antigen_concentration,bin,read_count
42492,SortSeq_bin1,lib1,A104C,1,AAAAACATCAGTTGGT,NaN,1,17
29591,SortSeq_bin1,lib1,A104C,1,AAAACACTATCTAGGA,NaN,1,36
59773,SortSeq_bin1,lib1,A104C,1,AAAATTCAAAATTATC,NaN,1,0
60801,SortSeq_bin1,lib1,A104C,1,AACAAAAGTGTATGTT,NaN,1,0
62532,SortSeq_bin1,lib1,A104C,1,AAGTTATGAATACCCT,NaN,1,0
...,...,...,...,...,...,...,...,...
7697113,TiteSeq_09_bin4,lib2,WT,0,TTTTTCATGTATATGC,0.0,4,0
7697148,TiteSeq_09_bin4,lib2,WT,0,TTTTTTAAAGTTCATA,0.0,4,0
7697151,TiteSeq_09_bin4,lib2,WT,0,TTTTTTACCTTTACCT,0.0,4,0
7697153,TiteSeq_09_bin4,lib2,WT,0,TTTTTTAGAAGCGAAG,0.0,4,0


Use total cell counts and total read counts in each concentration and bin to estimate the number of cells with each barcode

In [12]:
def normalize_read_count(df):
    library = df.library.iloc[0]
    sample = df["sample"].iloc[0]
    total_reads = df.read_count.sum()
    total_cells = barcode_runs.number_cells[(library, sample)]
    df["estimated_cell_count"] = total_cells * df.read_count / total_reads
    return df

df_barcodes = df_barcodes.groupby(["library", "sample"]).apply(normalize_read_count).reset_index(drop=True)
df_barcodes

,sample,library,variant,n_aa_substitutions,barcode,antigen_concentration,bin,read_count,estimated_cell_count
0,SortSeq_bin1,lib1,A104C,1,AAAAACATCAGTTGGT,NaN,1,17,0.610535
1,SortSeq_bin1,lib1,A104C,1,AAAACACTATCTAGGA,NaN,1,36,1.292898
2,SortSeq_bin1,lib1,A104C,1,AAAATTCAAAATTATC,NaN,1,0,0.000000
3,SortSeq_bin1,lib1,A104C,1,AACAAAAGTGTATGTT,NaN,1,0,0.000000
4,SortSeq_bin1,lib1,A104C,1,AAGTTATGAATACCCT,NaN,1,0,0.000000
...,...,...,...,...,...,...,...,...,...
7670155,TiteSeq_09_bin4,lib2,WT,0,TTTTTCATGTATATGC,0.0,4,0,0.000000
7670156,TiteSeq_09_bin4,lib2,WT,0,TTTTTTAAAGTTCATA,0.0,4,0,0.000000
7670157,TiteSeq_09_bin4,lib2,WT,0,TTTTTTACCTTTACCT,0.0,4,0,0.000000
7670158,TiteSeq_09_bin4,lib2,WT,0,TTTTTTAGAAGCGAAG,0.0,4,0,0.000000


## Barcode aggregation

The current $K_D$ estimation procedure does a separate estimate for each barcode, then computes the median of the estimated $\log K_D$ across barcodes for each variant.
We should instead estimate a single $K_D$ parameter for each variants. We will do this by aggregating read counts from all barcodes for a given variant.

In [13]:
df_variants = (
    df_barcodes
    .groupby(
        ["library", "variant", "n_aa_substitutions", "antigen_concentration", "bin", "sample"],
        dropna=False
    )
    .agg(
        {
            "read_count": "sum",
            "estimated_cell_count": "sum",
            "barcode": "count"
        }
    )
    .reset_index()
)
df_variants.sort_values(by=list(df_variants.columns), inplace=True)
df_variants

,library,variant,n_aa_substitutions,antigen_concentration,bin,sample,read_count,estimated_cell_count,barcode
0,lib1,A104C,1,0.000000e+00,1,TiteSeq_09_bin1,2663,1250.687875,25
1,lib1,A104C,1,0.000000e+00,2,TiteSeq_09_bin2,25,24.273527,25
2,lib1,A104C,1,0.000000e+00,3,TiteSeq_09_bin3,0,0.000000,25
3,lib1,A104C,1,0.000000e+00,4,TiteSeq_09_bin4,0,0.000000,25
4,lib1,A104C,1,1.000000e-13,1,TiteSeq_08_bin1,578,1494.790836,25
...,...,...,...,...,...,...,...,...,...
912835,lib2,Y94W R145M,2,1.000000e-06,4,TiteSeq_01_bin4,16,12.947315,1
912836,lib2,Y94W R145M,2,NaN,1,SortSeq_bin1,0,0.000000,1
912837,lib2,Y94W R145M,2,NaN,2,SortSeq_bin2,14,2.535604,1
912838,lib2,Y94W R145M,2,NaN,3,SortSeq_bin3,57,10.063316,1


## Filter missing concentrations

We filter missing concentrations for each barcode in the barcode data, and for each variant in the variant data. Missing means the sum of reads is zero.

In [14]:
def conc_filter_fn(df):
    return all(df.groupby("antigen_concentration", dropna=False).read_count.sum() > 0)

In [15]:
# make a 'sample_type' column, such that we can group sample type variants accross all bins when filtering for concentrations
df_variants["sample_type"] = df_variants["sample"].str.extract(r"^(TiteSeq|SortSeq)")
df_barcodes["sample_type"] = df_barcodes["sample"].str.extract(r"^(TiteSeq|SortSeq)")

In [16]:
df_barcodes = (
    df_barcodes
    .groupby(
        ["sample_type", "library", "variant", "n_aa_substitutions", "barcode"]
    )
    .filter(conc_filter_fn)
    .drop(columns=["sample_type"])
)
df_variants = (
    df_variants
    .groupby(
        ["sample_type", "library", "variant", "n_aa_substitutions"]
    )
    .filter(conc_filter_fn)
    .drop(columns=["sample_type"])
)

In [17]:
# plot the distribution of 'barcodes' values across the various each of the sample groups
df_variants.groupby(["sample", "library"]).barcode.describe()

count      mean         std  min  25%  50%   75%  \
sample          library                                                       
SortSeq_bin1    lib1     10938.0  8.470104  105.405655  1.0  1.0  1.0  15.0   
                lib2     11686.0  8.464060  114.128714  1.0  1.0  1.0  15.0   
SortSeq_bin2    lib1     10938.0  8.470104  105.405655  1.0  1.0  1.0  15.0   
                lib2     11686.0  8.464060  114.128714  1.0  1.0  1.0  15.0   
SortSeq_bin3    lib1     10938.0  8.470104  105.405655  1.0  1.0  1.0  15.0   
...                          ...       ...         ...  ...  ...  ...   ...   
TiteSeq_09_bin2 lib2     10200.0  9.548922  122.122509  1.0  1.0  1.0  17.0   
TiteSeq_09_bin3 lib1      9437.0  9.654551  113.434854  1.0  1.0  1.0  16.0   
                lib2     10200.0  9.548922  122.122509  1.0  1.0  1.0  17.0   
TiteSeq_09_bin4 lib1      9437.0  9.654551  113.434854  1.0  1.0  1.0  16.0   
                lib2     10200.0  9.548922  122.122509  1.0  1.0  1.0  17.0   

                             max  
sample          library           
SortSeq_bin1    lib1     10986.0  
                lib2     12299.0  
SortSeq_bin2    lib1     10986.0  
                lib2     12299.0  
SortSeq_bin3    lib1     10986.0  
...                          ...  
TiteSeq_09_bin2 lib2     12299.0  
TiteSeq_09_bin3 lib1     10986.0  
                lib2     12299.0  
TiteSeq_09_bin4 lib1     10986.0  
                lib2     12299.0  

[80 rows x 8 columns]

In [18]:
df_barcodes.to_csv(config['prepped_barcode_counts_file'], index=False)
df_variants.to_csv(config['prepped_variant_counts_file'], index=False)